# Proyecto Final - Deep Learning (1INF52)
## Entregable 2: Fine-tuning de T5 para generación de resúmenes

**Integrantes:**

Aarón Ulises Santillán Huamán - 20200445
Aarón Ulises Santillán Huamán - 20200445
Aarón Ulises Santillán Huamán - 20200445
Aarón Ulises Santillán Huamán - 20200445

> Este cuaderno implementa el pipeline de fine-tuning del modelo T5 sobre el dataset CNN/DailyMail
usando la librería 🤗 Hugging Face. Incluye carga de datos, preprocesamiento, entrenamiento,
evaluación con métricas ROUGE y guardado del modelo.

## 1. Importaciones y configuración inicial

In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
import os
import numpy as np
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)
import evaluate
import torch

# Verificar dispositivo disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo: {device}")

# Fijar semilla para reproducibilidad
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)


d:\Repositorios GitHub02\Deep-Learning-Project\venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dispositivo: cuda


## 2. Carga del dataset CNN/DailyMail

Usaremos la versión `3.0.0` del dataset `cnn_dailymail` disponible en Hugging Face Datasets.

In [3]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

## 3. Carga del tokenizer y del modelo T5

In [4]:
model_name = 't5-small'

tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)
model.gradient_checkpointing_enable()


## 4. Preprocesamiento de datos
Convertimos cada ejemplo en pares `(input_ids, labels)` para el modelo T5.

- Entrada: prefijo `summarize: ` + artículo
- Salida: `highlights` (resumen humano)


In [5]:
max_input_length = 256
max_target_length = 64

def preprocess_function(batch):
    inputs = ["summarize: " + doc for doc in batch['article']]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['highlights'],
            max_length=max_target_length,
            truncation=True,
            padding='max_length',
        )

    labels_ids = labels['input_ids']
    labels_ids = [
        [(lid if lid != tokenizer.pad_token_id else -100) for lid in label]
        for label in labels_ids
    ]
    model_inputs['labels'] = labels_ids
    return model_inputs


In [6]:
small_train = dataset['train'].shuffle(seed=seed).select(range(10000))  #Por variar
small_val = dataset['validation'].shuffle(seed=seed).select(range(1000))

tokenized_train = small_train.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names,
)
tokenized_val = small_val.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['validation'].column_names,
)
tokenized_train, tokenized_val

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 10000
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 1000
 }))

## 5. Definición de la métrica ROUGE

In [7]:
rouge = evaluate.load('rouge')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )
    # Dejar en porcentaje
    result = {k: round(v * 100, 2) for k, v in result.items()}
    return result


## 6. Configuración de entrenamiento (TrainingArguments + Trainer)

In [8]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.57.1
d:\Repositorios GitHub02\Deep-Learning-Project\venv311\Lib\site-packages\transformers\__init__.py


In [9]:
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)

output_dir = "./t5_cnn_summarization"

physical_batch_size = 1
num_epochs = 2
gradient_accumulation_steps = 8 


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=physical_batch_size,
    per_device_eval_batch_size=physical_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=True,
    fp16=True,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_epochs,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=False,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


C:\Users\USER\AppData\Local\Temp\ipykernel_30680\2820407950.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## 8. Entrenamiento del modelo

In [10]:
train_result = trainer.train()
trainer.save_model(output_dir)
train_result

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## 9. Evaluación en el conjunto de validación

In [ ]:
metrics = trainer.evaluate()
metrics

## 10. Ejemplos de resúmenes generados

In [ ]:
def generate_summary(article, max_new_tokens=128):
    # Construimos el prompt para T5
    input_text = "summarize: " + article
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        truncation=True,
        max_length=max_input_length,
    ).to(device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Tomamos un ejemplo del conjunto de validación original (no tokenizado)
example = dataset['validation'][0]
print('=== ARTÍCULO ORIGINAL ===')
print(example['article'][:1000], '...')

print('\n=== RESUMEN HUMANO (highlights) ===')
print(example['highlights'])

print('\n=== RESUMEN GENERADO POR EL MODELO ===')
generated = generate_summary(example['article'])
print(generated)
